In [ ]:
# Устанавливаем необходимые библиотеки
!pip install requests
!pip install beautifulsoup4

In [12]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import requests
import csv

In [ ]:
cookies_1 = {
    'session-cookie': '179de89f534327713e4cb0b280267f93923876c28415fa15a589c53167aa4cb029c12be69cf4abc34cef89b11b6438c4',
    'PHPSESSID': 'e0b4742f8c125cbe1b193d6c6d937062',
    'sputnik_session': '1701772010701|3',
}

headers_1 = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'ru,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'session-cookie=179de89f534327713e4cb0b280267f93923876c28415fa15a589c53167aa4cb029c12be69cf4abc34cef89b11b6438c4; PHPSESSID=e0b4742f8c125cbe1b193d6c6d937062; sputnik_session=1701772010701|3',
    'Referer': 'http://cikrf.ru/opendata/commitee_list.php',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.2471 YaBrowser/23.11.0.2471 Yowser/2.5 Safari/537.36',
}

cookies_2 = {
    'izbSession': '64b5b4e6-2f81-4b96-8c7e-e074bf77ddc0',
    'session-cookie': '179ef5084becc674cb4eb0b280267f9384308ec68e34b70f3eec65bab961a0ae57996adf4a546562c8843651c3afb13c',
    'izbFP': 'a289f9847ccd5273811bce9953e2894f',
    'JSESSIONID': 'b317e78f92f2a9363fa1963e3817',
}

headers_2 = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'ru,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    # 'Cookie': 'izbSession=64b5b4e6-2f81-4b96-8c7e-e074bf77ddc0; session-cookie=179ef5084becc674cb4eb0b280267f9384308ec68e34b70f3eec65bab961a0ae57996adf4a546562c8843651c3afb13c; izbFP=a289f9847ccd5273811bce9953e2894f; JSESSIONID=b317e78f92f2a9363fa1963e3817',
    'Origin': 'http://www.vybory.izbirkom.ru',
    'Referer': 'http://www.vybory.izbirkom.ru/region/izbirkom',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.2471 YaBrowser/23.11.0.2471 Yowser/2.5 Safari/537.36',
}


In [14]:
def fetch_data(url, headers, cookies):
    try:
        response = requests.get(url, headers=headers, cookies=cookies, verify=False, timeout=5)
        return (response, response.status_code)
    except requests.exceptions.ConnectionError:
        print("Connection refused")
    else:
        print(response.status_code)

In [15]:
def parse_and_write_from_xml_to_csv(xml_content, csv_filename):
    root = ET.fromstring(xml_content)

    # Открываем файл CSV для записи
    with open(csv_filename, 'w', encoding="utf-8", newline='') as csvfile:

        csv_writer = csv.writer(csvfile, delimiter=';')

        # Записываем заголовки CSV
        headers_of_csv_file = ["name"] + [child[0].text for child in root[0]]
        csv_writer.writerow(headers_of_csv_file)

        # Записываем данные из XML в CSV
        for parent in root:
            data_row = [parent.attrib["name"]] + [child[1].text if child[1] is not None else '' for child in parent]
            csv_writer.writerow(data_row)

        print(f"Данные успешно записаны в {csv_filename}")

Следующий парсер написан для сайта vybory.izbirkom.ru, так как сайт cikrf.ru не содержит данные о выборах раньше 2011 года. В таблицах избиркома заголовками являются названия регионов, а первая колонка содержит характеристики типа: "Число избирателей, внесенных в списки избирателей" и т.п. Однако это неудобное представление, поэтому пришлось условно транспонировать таблицу, чтобы характеристики были на месте заголовков.

In [16]:
def parse_and_write_from_html_to_csv(html_content, csv_filename):
    soup = BeautifulSoup(html_content, 'lxml')

    # Достаём заголовки будущего файла
    list_of_head = soup.find('table', id='fix-columns-table').find_all('td', class_='second-fix-col')
    native_headers = [list_of_head[i].text for i in range(0, len(list_of_head))]
    headers_of_csv_file = ["name"] + native_headers

    # Достаём "name" из изначальной таблицы
    soup_f_column = soup.find('table', id='fix-columns-table').find('thead').find_all('a')
    first_column = [soup.find('table', id='fix-columns-table').find('thead').find('b').text] + [soup_f_column[i].text for i in range(0, len(soup_f_column))]

    # Создаём и заполняем таблицу строчками из изначальной таблицы
    data = []
    data.append(first_column)

    new_soup = soup.find('table', id='fix-columns-table').find('tbody').find_all('tr')

    for tr in new_soup:
        rows = tr.find_all('b')
        if len(rows) != 0:
            pr_rows = [rows[i].text if rows[i] is not None else '' for i in range(0, len(rows))]
            data.append(pr_rows)

    # Открываем файл CSV для записи
    with open(csv_filename, 'w', encoding="utf-8", newline='') as csvfile:

        csv_writer = csv.writer(csvfile, delimiter=';')

        # Записываем заголовки CSV
        csv_writer.writerow(headers_of_csv_file)

        # Записываем данные из html в CSV
        for i in range(0, len(data[0])):
            data_row = []
            for j in range(0, len(data)):
                data_row.append(data[j][i])

            csv_writer.writerow(data_row)

        print(f"Данные успешно записаны в {csv_filename}")

Этот парсер написан специально для сводных результатов выборов президента РФ в 2004 году, так как код таблицы отличается по стилю от кодов остальных таблиц.

In [17]:
def parse_2004_elections_and_write_from_html_to_csv(html_content, csv_filename):
    soup = BeautifulSoup(html_content, 'lxml')
    soup_table = soup.find_all('table', class_='table-bordered')[1]

    # Достаём заголовки будущего файла
    h_soup = soup_table.find_all('td', class_='text-left')
    native_headers = [h_soup[i].find('nobr').text for i in range(0, len(h_soup))]
    headers_of_csv_file = ["name"] + native_headers

    # Достаём строчку с суммарными данными (по всей стране и не только)
    sum_row_soup = soup_table.find('td', style='vertical-align: baseline;').find_all('td', class_='text-right')
    sum_row = ["Сумма"] + [sum_row_soup[i].find('b').text for i in range(0, len(sum_row_soup))]

    # Достаём "name" из изначальной таблицы
    soup_f_column = soup_table.find_all('table', class_='table-bordered')[1].find_all('a')
    first_column = [soup_f_column[i].text for i in range(0, len(soup_f_column))]

    # Создаём и заполняем таблицу строчками из изначальной таблицы
    data = []
    data.append(first_column)
    table = soup_table.find_all('table', class_='table-striped')[1].find_all('tr')[1:]

    for tr in table:
        rows = tr.find_all('b')
        if len(rows) != 0:
            pr_rows = [rows[i].text if rows[i] is not None else '' for i in range(0, len(rows))]
            data.append(pr_rows)

    # Открываем файл CSV для записи
    with open(csv_filename, 'w', encoding="utf-8", newline='') as csvfile:

        csv_writer = csv.writer(csvfile, delimiter=';')

        # Записываем заголовки CSV
        csv_writer.writerow(headers_of_csv_file)

        # Записываем строчку с суммарными данными
        csv_writer.writerow(sum_row)

        # Записываем данные из html в CSV
        for i in range(0, len(data[0])):
            data_row = []
            for j in range(0, len(data)):
                data_row.append(data[j][i])

            csv_writer.writerow(data_row)

        print(f"Данные успешно записаны в {csv_filename}")

In [ ]:
# Ну и наконец создаем списки, которые определяют, в какой файл записываем данные по url-ссылке
list_of_ind_filenames = [(7, "Результаты_выборов_президента_2012"),
                         (15, "Результаты_выборов_в_думу_2011"),
                         (21, "Результаты_выборов_в_думу_2016"),
                         (23, "Результаты_выборов_в_думу_2016_по_фед"),
                         (41, "Результаты_выборов_президента_2018"),
                         (86, "Результаты_выборов_в_думу_2021_по_фед"),
                         (87, "Результаты_выборов_в_думу_2021")]
list_of_url_filenames = [("http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100095621&vrn=100100095619&region=0&global=1&sub_region=0&prver=0&pronetvd=0&vibid=100100095621&type=233","Результаты_выборов_в_думу_2003_по_фед"),
                         ("http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100021960186&vrn=100100021960181&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100021960186&type=233", "Результаты_выборов_в_думу_2007_по_фед"),
                         ("http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100022249920&vrn=100100022176412&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100022249920&type=227", "Результаты_выборов_президента_2008")]
uhhh_2004 = ("http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=1001000882951&vrn=1001000882950&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=1001000882951&type=227", "Результаты_выборов_президента_2004")

for pair in list_of_ind_filenames:
    csv_filename = f"{pair[1]}.csv"
    func_ret = fetch_data(f"http://www.cikrf.ru/opendata/{pair[0]}.xml", headers_1, cookies_1)
    if func_ret[1] == 200:
        parse_and_write_from_xml_to_csv(func_ret[0].content, csv_filename)

for pair in list_of_url_filenames:
    csv_filename = f"{pair[1]}.csv"
    func_ret = fetch_data(pair[0], headers_2, cookies_2)
    if func_ret[1] == 200:
        parse_and_write_from_html_to_csv(func_ret[0].text, csv_filename)


csv_filename = f"{uhhh_2004[1]}.csv"
func_ret = fetch_data(uhhh_2004[0], headers_2, cookies_2)
if func_ret == 200:
    parse_2004_elections_and_write_from_html_to_csv(func_ret[0].text, csv_filename)